### data

In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB
import os

### 모델 DER Aggregation 

In [42]:
def load_generation_data(include_files=None, date_filter=None):
    data_dir = "/Users/jangseohyun/Documents/workspace/symply/DER/data/generation"
    all_files = sorted([f for f in os.listdir(data_dir) if f.endswith('.csv')])

    if include_files is not None:
        for file in include_files:
            if file not in all_files:
                raise ValueError(f"파일을 찾을 수 없습니다: {file}")
        files_to_load = [f for f in all_files if f in include_files]
    else:
        files_to_load = all_files

    I = len(files_to_load)  
    T = 24 
    generation_data = np.zeros((I, T)) 

    loaded_files = []
    
    for idx, file in enumerate(files_to_load):
        file_path = os.path.join(data_dir, file)
        df = pd.read_csv(file_path)
        df.columns = df.columns.str.strip()

        date_col = "Date"
        hour_col = "Hour (Eastern Time, Daylight-Adjusted)"
        gen_col = "Electricity Generated"

        if any(col not in df.columns for col in [date_col, hour_col, gen_col]):
            print(f"{file}: 필요한 컬럼 없음. 스킵됨.")
            continue

        if date_filter:
            df = df[df[date_col] == date_filter]
            if df.empty:
                print(f"{file}: {date_filter} 데이터 없음. 스킵됨.")
                continue 

        df = df[df[hour_col].astype(str).str.match(r'^\d+$')]
        df["Time"] = df[hour_col].astype(int)
        df = df[df["Time"].between(0, 23)]

        for t in range(T):
            if t in df["Time"].values:
                generation_data[idx, t] = df[df["Time"] == t][gen_col].values[0]

        loaded_files.append(file)

    print(f"✅ 총 {I}개 파일을 불러왔습니다: {', '.join(loaded_files)}")

    return generation_data, I, T

def generate_randomized_generation(I, T, S, data, randomness_level):
    np.random.seed(7)

    noise_ranges = {
        "low": (0.8, 1.2),
        "medium": (0.5, 1.5),
        "high": (0.2, 1.8),
    }

    if randomness_level not in noise_ranges:
        raise ValueError("Invalid randomness level. Please choose 'low', 'medium', or 'high'.")

    low, high = noise_ranges[randomness_level]
    noise_factors = np.random.uniform(low, high, size=(I, T, S))

    generation_r = np.expand_dims(data, axis=-1) * noise_factors
    
    print(f"📊 데이터 Shape: I={I}, T={T}, S={S}")
    return generation_r

def plot_generation_data(generation_data, I):
    hours = np.arange(24)
    plt.figure(figsize=(15, 9))

    for i in range(I):
        plt.plot(hours, generation_data[i], marker='o', linestyle='-', alpha=0.7, label=f'Generator {i}')

    plt.xlabel("Hour")
    plt.ylabel("Electricity Generated (kWh)")
    plt.title("Hourly Electricity Generation for All Generators")
    plt.xticks(hours)  # 0~23 시간 설정
    plt.legend(loc="upper left", fontsize='small')

    plt.show()

def plot_randomized_generation(R, I, T, S):
    hours = np.arange(T)
    
    plt.figure(figsize=(15, 9))

    for i in range(I):
        plt.plot(hours, R[i, :, S], marker='o', linestyle='-', alpha=0.7, label=f'Generator {i}')

    plt.xlabel("Hour")
    plt.ylabel("Electricity Generated (kWh)")
    plt.title(f"Randomized Hourly Generation for Scenario {S}")
    plt.xticks(hours) 
    plt.legend(loc="upper left") 

    plt.show()
       
def plot_scenarios_for_generator(R, i):

    T = R.shape[1]
    S = R.shape[2] 
    hours = np.arange(T) 

    plt.figure(figsize=(15, 9))

    for s in range(S):
        plt.plot(hours, R[i, :, s], linestyle='-', alpha=0.6, label=f'Scenario {s+1}')

    plt.xlabel("Hour")
    plt.ylabel("Electricity Generated (kWh)")
    plt.title(f"Hourly Electricity Generation for Generator {i} Across All Scenarios")
    plt.xticks(hours)
    plt.legend(loc="upper left", fontsize='small', ncol=2)
    plt.show()

def generate_rt_scenarios(rt_da, S, randomness_level):

    rt_da["Time Stamp"] = pd.to_datetime(rt_da["Time Stamp"])
    nyc_rt = rt_da[rt_da["Name"] == "N.Y.C."].copy() 

    nyc_rt["Hour"] = nyc_rt["Time Stamp"].dt.floor("H")
    hourly_avg = nyc_rt.groupby("Hour")["LBMP ($/MWHr)"].mean().reset_index()
    price_hourly = hourly_avg["LBMP ($/MWHr)"].to_numpy()
    T = len(price_hourly)

    noise_ranges = {
        "low": (0.95, 1.05),
        "medium": (0.85, 1.15),
        "high": (0.7, 1.3),
    }

    if randomness_level not in noise_ranges:
        raise ValueError("Invalid randomness level. Choose from 'low', 'medium', 'high'.")

    low, high = noise_ranges[randomness_level]
    noise_factors = np.random.uniform(low, high, size=(T, S))

    P_RT = np.expand_dims(price_hourly, axis=-1) * noise_factors

    return P_RT

def plot_rt_scenarios(P_RT):
    T, S = P_RT.shape
    hours = np.arange(T)

    plt.figure(figsize=(15, 8))

    for s in range(S):
        plt.plot(hours, P_RT[:, s], linestyle='-', alpha=0.6, label=f"Scenario {s+1}")

    plt.xlabel("Hour")
    plt.ylabel("Price ($/MWHr)")
    plt.title("Real-Time Price Scenarios (Hourly Averaged)")
    plt.xticks(hours)
    plt.legend(loc="upper left", fontsize="small", ncol=2)

    plt.show()
    
only_profit = np.array(pd.read_csv("result/result_only_profit.csv"))
ny_da = pd.read_csv("/Users/jangseohyun/Documents/workspace/symply/DER/data/price/20220718da.csv")
ny_rt = pd.read_csv("/Users/jangseohyun/Documents/workspace/symply/DER/data/price/20220718rt.csv")
ny_da["Time Stamp"] = pd.to_datetime(ny_da["Time Stamp"])
ny_da["Hour"] = ny_da["Time Stamp"].dt.hour
nyc_data = ny_da[ny_da["Name"] == "N.Y.C."]
P_DA = nyc_data["LBMP ($/MWHr)"].to_numpy() * 1.3
P_PN = P_DA * 1.5

# plot_generation_data(generation_data, 10)
# plot_randomized_generation(R, I, T, 7)
# plot_scenarios_for_generator(R, 1)
# plot_rt_scenarios(P_RT)

include_files = ['1201.csv', '137.csv', '281.csv', '397.csv', '401.csv', '430.csv', '514.csv', '524.csv', '775.csv', '89.csv']
generation_data, I, T = load_generation_data(include_files, "2022-07-18")

S = 20
randomness_level = "medium"
R = generate_randomized_generation(I, T, S, generation_data, randomness_level)
P_RT = generate_rt_scenarios(ny_rt, S, randomness_level)

✅ 총 10개 파일을 불러왔습니다: 1201.csv, 137.csv, 281.csv, 397.csv, 401.csv, 430.csv, 514.csv, 524.csv, 775.csv, 89.csv
📊 데이터 Shape: I=10, T=24, S=20


In [43]:
agg = gp.Model("agg")
# agg.Params.OutputFlag = 0
agg.Params.MIPGap = 0.00001  # MIP gap을 1%로 설정


alpha = agg.addVars(T, vtype=GRB.CONTINUOUS, lb=0, name="alpha")
beta_plus = agg.addVars(T, S, vtype=GRB.CONTINUOUS, lb=0, name="beta_plus")
beta_minus = agg.addVars(T, S, vtype=GRB.CONTINUOUS, lb=0, name="beta_minus")

M = max(sum(R[i, t, s] for i in range(I)) for t in range(T) for s in range(S))
z = agg.addVars(T, S, vtype=GRB.BINARY, name="z")
prob = np.array([1 / S for s in range(S)])

agg.update()

obj = gp.quicksum(P_DA[t] * alpha[t] for t in range(T)) + gp.quicksum(
    prob[s] * (P_RT[t, s] * beta_plus[t, s] - P_PN[t] * beta_minus[t, s])
    for t in range(T)
    for s in range(S)
)

agg.setObjective(obj, GRB.MAXIMIZE)

Set parameter MIPGap to value 1e-05


In [44]:
for t in range(T):
    for s in range(S):
        agg.addConstr(
            gp.quicksum(R[i, t, s] for i in range(I)) - alpha[t]
            == beta_plus[t, s] - beta_minus[t, s]
        )

for t in range(T):
    for s in range(S):
        agg.addConstr(gp.quicksum(R[i, t, s] for i in range(I)) >= beta_plus[t, s])

for t in range(T):
    for s in range(S):
        agg.addConstr(beta_plus[t, s] <= M * z[t, s])
        agg.addConstr(beta_minus[t, s] <= M * (1 - z[t, s]))

agg.optimize()

if agg.status == GRB.OPTIMAL:
    print("Optimal solution found!")
    print(f"Objective value: {agg.objVal}")
else:
    print("No optimal solution found.")

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i9-13900K, instruction set [SSE2|AVX|AVX2]
Thread count: 24 physical cores, 32 logical processors, using up to 32 threads

Non-default parameters:
MIPGap  1e-05

Optimize a model with 1920 rows, 1464 columns and 3840 nonzeros
Model fingerprint: 0x20886b8a
Variable types: 984 continuous, 480 integer (480 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [3e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e-02, 2e+03]
Found heuristic solution: objective 1647313.0411
Presolve removed 1860 rows and 1403 columns
Presolve time: 0.02s
Presolved: 60 rows, 61 columns, 140 nonzeros
Found heuristic solution: objective 2107819.0797
Variable types: 41 continuous, 20 integer (20 binary)

Root relaxation: objective 2.107910e+06, 50 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bound

### 결과 분석

#### 수익 분석

In [45]:
# Day-ahead 수익 계산
total_da_profit_obj = 0
for t in range(T):
    total_da_profit_obj += P_DA[t] * alpha[t].x  

# Real-time 수익 계산
total_rt_profit_obj = 0
for t in range(T):
    for s in range(S):
        rt_profit_obj = P_RT[t, s] * beta_plus[t, s].x  
        total_rt_profit_obj += prob[s] * rt_profit_obj

# Penalty 비용 계산
total_penalty_cost_obj = 0
for t in range(T):
    for s in range(S):
        penalty_cost_obj = P_PN[t] * beta_minus[t, s].x
        total_penalty_cost_obj += prob[s] * penalty_cost_obj

# 총 시스템 이익 (목적 함수 기반)
total_system_profit_obj = total_da_profit_obj + total_rt_profit_obj - total_penalty_cost_obj

# 결과 출력
print(f"DA: {total_da_profit_obj:.2f}")
print(f"RT: {total_rt_profit_obj:.2f}")
print(f"Penalty: {total_penalty_cost_obj:.2f}")
print(f"목적 함수 기반 총 이익: {total_system_profit_obj:.2f}")

DA: 600925.44
RT: 1523566.31
Penalty: 16581.29
목적 함수 기반 총 이익: 2107910.46


#### 하루 aggregated 커밋량 분석

In [46]:
# 각 시간대 t에 대한 평균 alpha, beta+ 및 beta- 계산 및 출력
for t in range(T):
    avg_alpha = alpha[t].x  # alpha[t]의 최적화 값
    avg_beta_plus = sum(beta_plus[t, s].x for s in range(S)) / S  # beta_plus의 평균
    avg_beta_minus = sum(beta_minus[t, s].x for s in range(S)) / S  # beta_minus의 평균

    print(f"[시간 {t}] alpha: {avg_alpha:.3f}, beta+: {avg_beta_plus:.3f}, beta-: {avg_beta_minus:.3f}")

# alpha 값의 총합 출력
total_alpha = sum(alpha[t].x for t in range(T))
print(f"총 하루 commitment: {total_alpha:.3f}")

[시간 0] alpha: 0.000, beta+: 0.000, beta-: 0.000
[시간 1] alpha: 0.000, beta+: 0.127, beta-: 0.000
[시간 2] alpha: 0.132, beta+: 0.098, beta-: 0.000
[시간 3] alpha: 0.000, beta+: 0.000, beta-: 0.000
[시간 4] alpha: 0.543, beta+: 0.218, beta-: 0.001
[시간 5] alpha: 0.000, beta+: 0.826, beta-: 0.000
[시간 6] alpha: 7.763, beta+: 0.797, beta-: 0.176
[시간 7] alpha: 31.664, beta+: 10.233, beta-: 0.089
[시간 8] alpha: 84.867, beta+: 22.477, beta-: 0.670
[시간 9] alpha: 261.561, beta+: 24.095, beta-: 8.507
[시간 10] alpha: 657.547, beta+: 54.144, beta-: 6.932
[시간 11] alpha: 0.000, beta+: 893.379, beta-: 0.000
[시간 12] alpha: 0.000, beta+: 1263.720, beta-: 0.000
[시간 13] alpha: 0.000, beta+: 1766.632, beta-: 0.000
[시간 14] alpha: 1655.569, beta+: 169.851, beta-: 40.808
[시간 15] alpha: 0.000, beta+: 1178.700, beta-: 0.000
[시간 16] alpha: 796.334, beta+: 143.320, beta-: 5.165
[시간 17] alpha: 0.000, beta+: 920.147, beta-: 0.000
[시간 18] alpha: 0.000, beta+: 692.545, beta-: 0.000
[시간 19] alpha: 391.276, beta+: 54.208, beta-

### 사후정산

In [47]:
only_value = pd.read_csv('result/result_only_obj.csv').values
only_profit = pd.read_csv('result/result_only_profit.csv').values
surplus = agg.objVal - only_value[0] #

I, T, S = R.shape

R_proportion = R / R.sum(axis=1, keepdims=True) 
R_proportion = np.nan_to_num(R_proportion) 

R_proportion_P = np.multiply(R_proportion, P_DA[:, np.newaxis])

R_weighted = R_proportion_P.sum(axis=(1, 2)) 

R_weighted_normalized = R_weighted / R_weighted.sum() 

surplus_distribution = surplus * R_weighted_normalized  

final_profit = only_profit.flatten() + surplus_distribution

print("Surplus 분배 결과:")
for i, value in enumerate(surplus_distribution):
    print(f"[{i}]: {value:.2f}")

print("\n최종 Profit:")
for i, (profit, only) in enumerate(zip(final_profit, only_profit.flatten())):
    increase_percentage = ((profit - only) / only) * 100
    print(f"[{i}] {profit:.2f} ({increase_percentage:.2f}%)")

Surplus 분배 결과:
[0]: 2321.30
[1]: 2382.17
[2]: 2333.54
[3]: 2278.09
[4]: 2330.22
[5]: 2344.31
[6]: 2402.36
[7]: 2422.39
[8]: 2239.04
[9]: 2356.74

최종 Profit:
[0] 204900.53 (1.15%)
[1] 340956.97 (0.70%)
[2] 192391.08 (1.23%)
[3] 54396.02 (4.37%)
[4] 375408.27 (0.62%)
[5] 95491.41 (2.52%)
[6] 93490.16 (2.64%)
[7] 458364.45 (0.53%)
[8] 132906.12 (1.71%)
[9] 159605.46 (1.50%)


### 결과 저장

In [48]:
alpha_df = pd.DataFrame({
    'alpha': [alpha[t].x for t in range(T)]
})
alpha_df.to_csv('result/result_base_alpha.csv', index=False)

# # beta_plus 저장
# beta_plus_df = pd.DataFrame(
#     [[beta_plus[t,s].x for s in range(S)] 
#      for t in range(T)],
#     columns=[f'S{s}' for s in range(S)]
# )
# beta_plus_df.to_csv('result_beta_plus.csv', index=False)

# # beta_minus 저장
# beta_minus_df = pd.DataFrame(
#     [[beta_minus[t,s].x for s in range(S)] 
#      for t in range(T)],
#     columns=[f'S{s}' for s in range(S)]
# )
# beta_minus_df.to_csv('result_beta_minus.csv', index=False)

In [49]:
agg_profit_value = pd.DataFrame({'agg_profit_value': final_profit})
agg_profit_value.to_csv('result/result_agg_profit.csv', index=False)

agg_obj = pd.DataFrame({'agg_obj': [agg.objVal]})
agg_obj.to_csv('result/result_agg_obj.csv', index=False)